In [26]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# 1. Exploración del conjunto de datos:

In [27]:
#Cargo el fichero, sin que aparezca "Unnamed: 0".
df = pd.read_csv('Spotify_Youtube.csv', index_col = 0)
df.head()

,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [28]:
#Últimas 5 filas del dataframe
df.tail(5)

,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
20713,SICK LEGEND,https://open.spotify.com/artist/3EYY5FwDkHEYLw...,JUST DANCE HARDSTYLE,JUST DANCE HARDSTYLE,single,spotify:track:0RtcKQGyI4hr8FgFH1TuYG,0.582,0.926,5.0,-6.344,0.0328,0.44800,0.0000,0.0839,0.6580,90.002,94667.0,https://www.youtube.com/watch?v=5SHmKFKlNqI,JUST DANCE HARDSTYLE,SICK LEGEND - Topic,71678.0,1113.0,0.0,Provided to YouTube by Routenote\n\nJUST DANCE...,True,True,9227144.0
20714,SICK LEGEND,https://open.spotify.com/artist/3EYY5FwDkHEYLw...,SET FIRE TO THE RAIN HARDSTYLE,SET FIRE TO THE RAIN HARDSTYLE,single,spotify:track:3rHvPA8lUnPBkaLyPOc0VV,0.531,0.936,4.0,-1.786,0.1370,0.02800,0.0000,0.0923,0.6570,174.869,150857.0,https://www.youtube.com/watch?v=ocTH6KxllDQ,SET FIRE TO THE RAIN HARDSTYLE,SICK LEGEND - Topic,164741.0,2019.0,0.0,Provided to YouTube by Routenote\n\nSET FIRE T...,True,True,10898176.0
20715,SICK LEGEND,https://open.spotify.com/artist/3EYY5FwDkHEYLw...,OUTSIDE HARDSTYLE SPED UP,OUTSIDE HARDSTYLE SPED UP,single,spotify:track:4jk00YxPtPbhvHJE9N4ddv,0.443,0.830,4.0,-4.679,0.0647,0.02430,0.0000,0.1540,0.4190,168.388,136842.0,https://www.youtube.com/watch?v=5wFhE-HY0hg,OUTSIDE HARDSTYLE SPED UP,SICK LEGEND - Topic,35646.0,329.0,0.0,Provided to YouTube by Routenote\n\nOUTSIDE HA...,True,True,6226110.0
20716,SICK LEGEND,https://open.spotify.com/artist/3EYY5FwDkHEYLw...,ONLY GIRL HARDSTYLE,ONLY GIRL HARDSTYLE,single,spotify:track:5EyErbpsugWliX006eTDex,0.417,0.767,9.0,-4.004,0.4190,0.35600,0.0184,0.1080,0.5390,155.378,108387.0,https://www.youtube.com/watch?v=VMFLbFRNCn0,ONLY GIRL HARDSTYLE,SICK LEGEND - Topic,6533.0,88.0,0.0,Provided to YouTube by Routenote\n\nONLY GIRL ...,True,True,6873961.0
20717,SICK LEGEND,https://open.spotify.com/artist/3EYY5FwDkHEYLw...,MISS YOU HARDSTYLE,MISS YOU HARDSTYLE,single,spotify:track:6lOn0jz1QpjcWeXo1oMm0k,0.498,0.938,6.0,-4.543,0.1070,0.00277,0.9110,0.1360,0.0787,160.067,181500.0,https://www.youtube.com/watch?v=zau0dckCFi0,MISS YOU HARDSTYLE,SICK LEGEND - Topic,158697.0,2484.0,0.0,Provided to YouTube by Routenote\n\nMISS YOU H...,True,True,5695584.0


In [29]:
#10 filas aleatorias del dataframe
df.sample(10)


,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
2223,Paul McCartney,https://open.spotify.com/artist/4STHEaNw4mPZ2t...,Every Night - 2011 Remaster,McCartney (Archive Collection),album,spotify:track:7zE7VClzAEYa9nUDC950pT,0.605,0.404,4.0,-12.366,0.0256,0.58800,0.001430,0.1040,0.867,94.285,153413.0,https://www.youtube.com/watch?v=aSHkH_uMmtY,Every Night (2011 Remaster),Paul McCartney - Topic,714970.0,10881.0,64.0,Provided to YouTube by Universal Music Group\n...,True,True,12787820.0
16401,Ezhel,https://open.spotify.com/artist/6LnJKrtFnTEGdb...,Bul Beni,Bul Beni,single,spotify:track:1riBwVUqVsk5RGJ9iLN8cG,0.797,0.540,6.0,-7.862,0.2680,0.52800,0.000000,0.1020,0.348,104.001,154173.0,https://www.youtube.com/watch?v=sxVYkUVtih4,Ezhel - Bul Beni,Ezhel,79751284.0,503292.0,16368.0,►'Bul Beni' OUT NOW: https://stageart.lnk.to/B...,True,True,107969332.0
3861,TLC,https://open.spotify.com/artist/0TImkz4nPqjegt...,Baby-Baby-Baby,Ooooooohhh... On The TLC Tip,album,spotify:track:1zTuB57LYZa7xu7KUH8kF0,0.679,0.597,11.0,-8.601,0.0439,0.09130,0.000673,0.0487,0.900,184.174,315040.0,https://www.youtube.com/watch?v=Q2H6pTtEVlo,TLC - Baby-Baby-Baby (Official Video),TLCVEVO,32571330.0,228662.0,10212.0,"""Baby-Baby-Baby"" by TLC\nListen to TLC: https:...",True,True,17810889.0
14055,Steve Lacy,https://open.spotify.com/artist/57vWImR43h4CaD...,Infrunami,The Lo-Fis,album,spotify:track:0f8eRy9A0n6zXpKSHSCAEp,0.826,0.299,9.0,-7.721,0.0446,0.31000,0.095200,0.0758,0.215,97.012,178328.0,https://www.youtube.com/watch?v=FmtEkkbCyLQ,Steve Lacy - Infrunami,MASSHENDRIX,114210.0,5131.0,114.0,I decided to put together another Steve Lacy v...,True,True,116437995.0
13453,Girls' Generation,https://open.spotify.com/artist/0Sadg1vgvaPqGT...,Lion Heart,Lion Heart - The 5th Album,album,spotify:track:405SQUJdQut02dxtuQ0CZ3,0.754,0.788,2.0,-3.175,0.0322,0.12900,0.002740,0.0775,0.961,124.992,224258.0,https://www.youtube.com/watch?v=nVCubhQ454c,Girls' Generation 소녀시대 'Lion Heart' MV,SMTOWN,135174802.0,1201912.0,70410.0,Girls’ Generation’s regular 5th album which co...,True,True,52655129.0
3896,Herbert von Karajan,https://open.spotify.com/artist/5zCaQxjl110XTr...,"Symphony No. 9 in E Minor, Op. 95, B. 178 ""Fro...","Dvorák: Symphony No.9 , Op.95, B. 178 ""From th...",album,spotify:track:6nqHzwOdGIaX57U6VU6kMO,0.249,0.163,4.0,-15.638,0.0351,0.90100,0.741000,0.1240,0.112,136.037,685000.0,https://www.youtube.com/watch?v=P_1N6_O254g,"Dvořák: Symphony No. 9 ""From the New World"" / ...",Berliner Philharmoniker,6898121.0,96713.0,3758.0,Full-length concert: http://www.digitalconcert...,False,True,8003013.0
5455,Los Acosta,https://open.spotify.com/artist/1sxekzjeHJaqkZ...,Tonto corazón,Tonto Corazón,album,spotify:track:0fDgvO2fI6q3ODozmsHXwD,0.595,0.945,2.0,-1.675,0.0284,0.00916,0.000068,0.2050,0.619,103.759,206640.0,https://www.youtube.com/watch?v=uzDxqUciqEg,LOS ACOSTA - Tonto corazón Nueva versión,LosAcostaOficial,13296558.0,52897.0,3577.0,#tontocorazon2019 @RicardoAcostaLosAcosta \n\n...,False,False,33455106.0
20091,Guilherme & Benuto,https://open.spotify.com/artist/6m6e7D2TnV0aYM...,Batom Vermelho,Batom Vermelho,single,spotify:track:72az3XEy1rALhVuoepsFpl,0.654,0.901,11.0,-3.212,0.1310,0.56900,0.000000,0.1320,0.858,163.981,171244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23864819.0
14193,Filipe Ret,https://open.spotify.com/artist/7gJN8W0589FisS...,Corte Americano,Imaterial (Deluxe),album,spotify:track:68qqVn5kHBv4zc45SDGLnh,0.728,0.665,5.0,-5.519,0.0595,0.55500,0.000000,0.1400,0.473,129.055,139535.0,https://www.youtube.com/watch?v=ykojPtsfK64,Filipe Ret - Corte Americano part. L7NNON,Filipe Ret,91988773.0,878334.0,11034.0,Ouça Imaterial Deluxe que está disponível nas ...,True,True,108857024.0
5337,Too $hort,https://open.spotify.com/artist/4sb7rZNN93BSS6...,Just Another Day,Get In Where You F

In [30]:
#Numero de ántas filas y columnas
df.shape

(20718, 27)

In [31]:
#tipos de los datos de cada columna del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20718 entries, 0 to 20717
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist            20718 non-null  object 
 1   Url_spotify       20718 non-null  object 
 2   Track             20718 non-null  object 
 3   Album             20718 non-null  object 
 4   Album_type        20718 non-null  object 
 5   Uri               20718 non-null  object 
 6   Danceability      20716 non-null  float64
 7   Energy            20716 non-null  float64
 8   Key               20716 non-null  float64
 9   Loudness          20716 non-null  float64
 10  Speechiness       20716 non-null  float64
 11  Acousticness      20716 non-null  float64
 12  Instrumentalness  20716 non-null  float64
 13  Liveness          20716 non-null  float64
 14  Valence           20716 non-null  float64
 15  Tempo             20716 non-null  float64
 16  Duration_ms       20716 non-null  float64
 17

In [32]:
#valores nulos tenemos por columna
df.isnull().sum()

Artist                0
Url_spotify           0
Track                 0
Album                 0
Album_type            0
Uri                   0
Danceability          2
Energy                2
Key                   2
Loudness              2
Speechiness           2
Acousticness          2
Instrumentalness      2
Liveness              2
Valence               2
Tempo                 2
Duration_ms           2
Url_youtube         470
Title               470
Channel             470
Views               470
Likes               541
Comments            569
Description         876
Licensed            470
official_video      470
Stream              576
dtype: int64

In [33]:
# Verificar todas las ocurrencias duplicadas en el DataFrame
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
20713    False
20714    False
20715    False
20716    False
20717    False
Length: 20718, dtype: bool

In [34]:
df.duplicated().sum()

0

In [35]:
#estadisticos column numericas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Danceability,20716.0,6.197775e-01,1.652724e-01,0.000000,5.180000e-01,6.370000e-01,7.402500e-01,9.750000e-01
Energy,20716.0,6.352503e-01,2.141468e-01,0.000020,5.070000e-01,6.660000e-01,7.980000e-01,1.000000e+00
Key,20716.0,5.300348e+00,3.576449e+00,0.000000,2.000000e+00,5.000000e+00,8.000000e+00,1.100000e+01
Loudness,20716.0,-7.671680e+00,4.632749e+00,-46.251000,-8.858000e+00,-6.536000e+00,-4.931000e+00,9.200000e-01
Speechiness,20716.0,9.645601e-02,1.119600e-01,0.000000,3.570000e-02,5.050000e-02,1.030000e-01,9.640000e-01
Acousticness,20716.0,2.915353e-01,2.862988e-01,0.000001,4.520000e-02,1.930000e-01,4.772500e-01,9.960000e-01
Instrumentalness,20716.0,5.596156e-02,1.932620e-01,0.000000,0.000000e+00,2.405000e-06,4.630000e-04,1.000000e+00
Liveness,20716.0,1.935210e-01,1.685309e-01,0.014500,9.410000e-02,1.250000e-01,2.370000e-01,1.000000e+00
Valence,20716.0,5.298533e-01,2.454408e-01,0.000000,3.390000e-01,5.370000e-01,7.262500e-01,9.930000e-01
Tempo,20716.0,1.206383e+02,2.957902e+01,0.000000,9.700200e+01,1.199650e+02,1.399350e+02,2.433720e+02


In [36]:
#estadisticos column categorics
df.describe(include='O').T

,count,unique,top,freq
Artist,20718,2079,Gorillaz,10
Url_spotify,20718,2079,https://open.spotify.com/artist/3AA28KZvwAUcZu...,10
Track,20718,17841,El Ultimo Adiós - Varios Artistas Version,24
Album,20718,11937,Greatest Hits,30
Album_type,20718,3,album,14926
Uri,20718,18862,spotify:track:0tzixmHNQfE6S6SirSToxW,24
Url_youtube,20248,18154,https://www.youtube.com/watch?v=GqOvxbFLwjY,18
Title,20248,18146,Color Esperanza 2020 - Various Artists (Offici...,18
Channel,20248,6714,T-Series,238
Description,19842,17395,,114


# 2. Preparación de los datos:

In [37]:
#nombres a minusculas
# vamos a iterar por todas las columnas del DataFrame y a cada una de ellas 
#la pondremos en minúsula. 
nuevas_columnas = {columna: columna.lower() for columna in df.columns}

# comprobamos que hemos creado el diccionario correctamente
nuevas_columnas

{'Artist': 'artist',
 'Url_spotify': 'url_spotify',
 'Track': 'track',
 'Album': 'album',
 'Album_type': 'album_type',
 'Uri': 'uri',
 'Danceability': 'danceability',
 'Energy': 'energy',
 'Key': 'key',
 'Loudness': 'loudness',
 'Speechiness': 'speechiness',
 'Acousticness': 'acousticness',
 'Instrumentalness': 'instrumentalness',
 'Liveness': 'liveness',
 'Valence': 'valence',
 'Tempo': 'tempo',
 'Duration_ms': 'duration_ms',
 'Url_youtube': 'url_youtube',
 'Title': 'title',
 'Channel': 'channel',
 'Views': 'views',
 'Likes': 'likes',
 'Comments': 'comments',
 'Description': 'description',
 'Licensed': 'licensed',
 'official_video': 'official_video',
 'Stream': 'stream'}

In [38]:
# una vez que tenemos el diccionario vamos a aplicar el método rename
df.rename(columns = nuevas_columnas, inplace = True)

# comprobamos que se han cambiado los nombres de las columnas
df.head()

,artist,url_spotify,track,album,album_type,uri,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,url_youtube,title,channel,views,likes,comments,description,licensed,official_video,stream
0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [39]:
#las variables ya estan con el tipo correcto


In [40]:
#pasamos los títulos a minúscula
# Definir la función para convertir a minúsculas
def to_lowercase(text):
    return text.lower() if isinstance(text, str) else text

In [41]:
# Aplicar la función a las columnas "title" y "album"
df['title'] = df['title'].apply(to_lowercase)
df['album'] = df['album'].apply(to_lowercase)
df['track'] = df['track'].apply(to_lowercase)

In [42]:
# comprobamos que se han cambiado los nombres de las columnas
df.head()

,artist,url_spotify,track,album,album_type,uri,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,url_youtube,title,channel,views,likes,comments,description,licensed,official_video,stream
0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,feel good inc.,demon days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,gorillaz - feel good inc. (official video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,rhinestone eyes,plastic beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,https://www.youtube.com/watch?v=yYDmaexVHic,gorillaz - rhinestone eyes [storyboard film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,new gold (feat. tame impala and bootie brown),new gold (feat. tame impala and bootie brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,https://www.youtube.com/watch?v=qJa-VFwPpYA,gorillaz - new gold ft. tame impala & bootie b...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,on melancholy hill,plastic beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,https://www.youtube.com/watch?v=04mfKJWDSzI,gorillaz - on melancholy hill (official video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,clint eastwood,gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,https://www.youtube.com/watch?v=1V_xRb0x9aw,gorillaz - clint eastwood (official video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [43]:
df['artist'].unique()

array(['Gorillaz', 'Red Hot Chili Peppers', '50 Cent', ..., 'LE SSERAFIM',
       'ThxSoMch', 'SICK LEGEND'], dtype=object)

In [68]:
#funciona para reemplazar los $ por S
def reempl_s (df):
    df = df.replace("$", "s")
    return df

In [69]:
# Aplicar la función a la columna "artist"
df['artist'] = df['artist'].apply(reempl_s)

# 3. Filtrado de datos:

In [60]:
#valores unicos album_type
df['album_type'].unique()

array(['album', 'single', 'compilation'], dtype=object)

In [70]:
# Encontrar valores únicos de la columna "album_type"
unique_album_types = df['album_type'].unique()
print("Valores únicos de 'album_type':", unique_album_types)

# Crear un diccionario para almacenar los DataFrames y los conteos
df_by_album_type = {}
counts_by_album_type = {}
# Crear DataFrames separados para cada tipo de "album_type" y contar las canciones
for album_type in unique_album_types:
    data = df[df['album_type'] == album_type]
    df_by_album_type[album_type] = data
    counts_by_album_type[album_type] = len(data)

Valores únicos de 'album_type': ['album' 'single' 'compilation']


In [71]:
# Imprimir el número de canciones en cada tipo de "album_type"
for album_type, count in counts_by_album_type.items():
    print(f"Número de canciones en '{album_type}': {count}")

Número de canciones en 'album': 14926
Número de canciones en 'single': 5004
Número de canciones en 'compilation': 788


In [72]:
df_by_album_type

{'album':                artist                                        url_spotify  \
 0            Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
 1            Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
 3            Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
 4            Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
 5            Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
 ...               ...                                                ...   
 20663          AYLIVA  https://open.spotify.com/artist/2rEVnwCPBeGkWM...   
 20664          AYLIVA  https://open.spotify.com/artist/2rEVnwCPBeGkWM...   
 20665          AYLIVA  https://open.spotify.com/artist/2rEVnwCPBeGkWM...   
 20673  Grupo Frontera  https://open.spotify.com/artist/6XkjpgcEsYab50...   
 20683       Jung Kook  https://open.spotify.com/artist/6HaGTQPmzraVma...   
 
                                          track           album a

In [73]:
df_album = pd.DataFrame(df_by_album_type['album'])

In [74]:
df_single = pd.DataFrame(df_by_album_type['single'])

In [75]:
df_compilation = pd.DataFrame(df_by_album_type['compilation'])

In [76]:
# Agrupar por artista y calcular la media de visitas y likes del dataframe de los albumes
df_album.columns

Index(['artist', 'url_spotify', 'track', 'album', 'album_type', 'uri',
       'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'url_youtube', 'title', 'channel', 'views', 'likes',
       'comments', 'description', 'licensed', 'official_video', 'stream'],
      dtype='object')

In [84]:
# Agrupar por artista y calcular la media y desviación estándar
ests_artist = df_album.groupby('artist').agg({
    'danceability': ['mean', 'std'],
    'acousticness': ['mean', 'std'],
    'speechiness': ['mean', 'std']
}).reset_index()

# Convertir cada métrica en un DataFrame y ordenar por media
danceability_df = pd.DataFrame(ests_artist['danceability']).sort_values(by='mean', ascending=False)
acousticness_df = pd.DataFrame(ests_artist['acousticness']).sort_values(by='mean', ascending=False)
speechiness_df = pd.DataFrame(ests_artist['speechiness']).sort_values(by='mean', ascending=False)


In [88]:
# Mostrar los resultados como DF
print("Estadísticas por artista para danceability:")
danceability_df


Estadísticas por artista para danceability:


,mean,std
1003,0.922333,0.014503
1261,0.911500,0.030406
848,0.886000,0.015556
204,0.885000,0.043782
887,0.881000,NaN
...,...,...
1661,0.142067,0.042212
1341,0.137980,0.063706
1342,0.133533,0.061768
1897,0.090262,0.163895


In [89]:

print("\nEstadísticas por artista para acousticness:")
acousticness_df



Estadísticas por artista para acousticness:


,mean,std
351,0.993714,0.001380
1509,0.993444,0.004902
601,0.989500,0.005237
506,0.989500,0.007863
607,0.989286,0.004071
...,...,...
1879,0.000220,0.000237
1254,0.000166,0.000191
947,0.000132,0.000158
963,0.000083,NaN


In [90]:

print("\nEstadísticas por artista para speechiness:")
speechiness_df


Estadísticas por artista para speechiness:


,mean,std
1626,0.952300,0.008845
181,0.945300,0.011879
1601,0.939300,0.015833
1698,0.919300,0.023542
613,0.906200,0.048960
...,...,...
1877,0.027400,NaN
1476,0.027071,0.004159
347,0.027057,0.001265
1478,0.025300,NaN


In [77]:
# Agrupar por artista y calcular la media de visitas y likes del dataframe de los albumes
artist_stats = df_album.groupby('artist').agg({
    'views': 'mean',
    'likes': 'mean'
}).reset_index()

print("Estadísticas por artista:")
print(artist_stats)

Estadísticas por artista:
                 artist         views         likes
0              (G)I-DLE  1.317374e+08  1.987726e+06
1                *NSYNC  1.027833e+08  4.233423e+05
2             070 Shake  4.125686e+06  1.165698e+05
3                   112  2.383860e+07  1.599430e+05
4     11:11 Music Group  4.979400e+04  1.173000e+03
...                 ...           ...           ...
1984          will.i.am  6.523732e+08  3.101684e+06
1985     Ángela Aguilar  1.524076e+08  1.133840e+06
1986               Ñejo  6.946546e+07  3.225730e+05
1987         Ñengo Flow  1.016329e+08  7.513430e+05
1988           Øneheart  4.037900e+04  1.353000e+03

[1989 rows x 3 columns]


In [101]:
# Obtener los 10 artistas con más visitas
top_10_visitas = artist_stats.sort_values(by='views', ascending=False).head(10)

# Obtener los 10 artistas con más likes
top_10_likes = artist_stats.sort_values(by='likes', ascending=False).head(10)

In [104]:
print(f'el Top de visitas es\n: {top_10_visitas}')
print('----------------------------------------')
print(f'el Top de likes es {top_10_likes}')

el Top de visitas es
:              artist         views         likes
1267             MØ  3.324835e+09  1.621108e+07
501      Ed Sheeran  1.715103e+09  9.220239e+06
1100     Luis Fonsi  1.654743e+09  9.503034e+06
397        DJ Snake  1.547144e+09  1.027126e+07
355       CoComelon  1.460167e+09  4.171391e+06
915      Katy Perry  1.454552e+09  6.942745e+06
1296  Natti Natasha  1.379763e+09  5.663424e+06
889   Justin Bieber  1.197992e+09  8.199640e+06
1168    Mark Ronson  1.034353e+09  4.870958e+06
1597   Shawn Mendes  1.032747e+09  8.282233e+06
----------------------------------------
el Top de likes es              artist         views         likes
138             BTS  7.022951e+08  1.669447e+07
1267             MØ  3.324835e+09  1.621108e+07
397        DJ Snake  1.547144e+09  1.027126e+07
136       BLACKPINK  4.126891e+08  9.868695e+06
1100     Luis Fonsi  1.654743e+09  9.503034e+06
185   Billie Eilish  5.966131e+08  9.436989e+06
501      Ed Sheeran  1.715103e+09  9.220239e+06
50   

In [92]:
# Filtrar para quedarnos solo con las colaboraciones (track que contiene "feat")
df_colaboracion = df[df['track'].str.contains('feat', case=False, na=False)]
df_colaboracion

,artist,url_spotify,track,album,album_type,uri,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,url_youtube,title,channel,views,likes,comments,description,licensed,official_video,stream
2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,new gold (feat. tame impala and bootie brown),new gold (feat. tame impala and bootie brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,-3.930,0.0522,0.04250,0.046900,0.1160,0.551,108.014,215150.0,https://www.youtube.com/watch?v=qJa-VFwPpYA,gorillaz - new gold ft. tame impala & bootie b...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,63063467.0
6,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,new gold (feat. tame impala and bootie brown) ...,new gold (feat. tame impala and bootie brown) ...,single,spotify:track:2c3KCGq6UojB2c8UAFrRON,0.716,0.897,4.0,-7.185,0.0629,0.01200,0.262000,0.3250,0.358,127.030,274142.0,https://www.youtube.com/watch?v=BONNm0F7Tto,"gorillaz - new gold ft. tame impala, bootie br...",Dom Dolla,451996.0,11686.0,241.0,"Gorillaz 'New Gold' ft. Tame Impala, Bootie Br...",False,True,10666154.0
7,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,she's my collar (feat. kali uchis),humanz (deluxe),album,spotify:track:3lIxtCaROdRDuTnNBDm3n2,0.726,0.815,11.0,-5.886,0.0313,0.00799,0.081000,0.1120,0.462,140.158,209560.0,https://www.youtube.com/watch?v=f8NwLXYIHS4,gorillaz - she's my collar [hq],SalvaMuñox,1010982.0,17675.0,260.0,𝐁̲𝐎̲𝐍̲𝐔̲𝐒̲:̲ Hu̳ma̳n̳z [̲̠̲𝐃̲̠̲𝐄̲̠̲𝐅̲̠̲𝐈̲̠̲𝐍̲̠...,False,False,159605929.0
8,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,cracker island (feat. thundercat),cracker island (feat. thundercat),single,spotify:track:2W3ZpQg9i6lE6kmHbcdu9N,0.741,0.913,2.0,-3.340,0.0465,0.00343,0.103000,0.3250,0.643,120.012,213750.0,https://www.youtube.com/watch?v=S03T47hapAc,gorillaz - cracker island ft. thundercat (offi...,Gorillaz,24459820.0,739527.0,20296.0,Listen to Cracker Island: https://gorillaz.lnk...,True,True,42671901.0
27,50 Cent,https://open.spotify.com/artist/3q7HBObVc0L8jN...,is this love ('09) (feat. 50 cent),curtain call 2,album,spotify:track:7rK0iM6jwuo1oizdx8suQA,0.853,0.776,8.0,-4.221,0.2610,0.07140,0.000000,0.1810,0.659,94.028,212013.0,https://www.youtube.com/watch?v=Zl64MRAqQxk,"eminem, 50 cent - is this love ('09) [visualizer]",EminemVEVO,5289821.0,245935.0,9008.0,"Listen to ""Curtain Call 2"": \nhttps://Eminem.l...",True,True,28529840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20683,Jung Kook,https://open.spotify.com/artist/6HaGTQPmzraVma...,left and right (feat. jung kook of bts),charlie,album,spotify:track:5Odq8ohlgIbQKMZivbWkEo,0.881,0.592,2.0,-4.898,0.0324,0.61900,0.000013,0.0901,0.719,101.058,154487.0,https://www.youtube.com/watch?v=a7GITgqwDVg,charlie puth - left and right (feat. jung kook...,Charlie Puth,293367976.0,8016899.0,540887.0,Charlie Puth - Left And Right (feat. Jung Kook...,True,True,NaN
20685,Jung Kook,https://open.spotify.com/artist/6HaGTQPmzraVma...,left and right (feat. jung kook of bts) - acap...,left and right (feat. jung kook of bts),single,spotify:track:0lWsA6TPYDRWenhCJrbfYu,0.571,0.169,9.0,-8.628,0.0554,0.75300,0.000000,0.0975,0.742,202.365,154487.0,https://www.youtube.com/watch?v=a7GITgqwDVg,charlie puth - left and right (feat. jung kook...,Charlie Puth,293367976.0,8016899.0,540887.0,Charlie Puth - Left And Right (feat. Jung Kook...,True,True,NaN
20686,Jung Kook,https://open.spotify.com/artist/6HaGTQPmzraVma...,left and right (feat. jung kook of bts) - sam ...,left and right (feat. jung kook of bts) [sam f...,single,spotify:track:17IVw95dOZSIiq7dz4UWaD,0.773,0.836,2.0,-5.366,0.0724,0.02950,0.000448,0.0914,0.614,122.006,162541.0,https://www.youtube.com/watch?v=aJ3_PzuU_jY,charlie puth - left and right (feat. jung kook...,Charlie Puth,813109.0,63116.0,2261.0,New album 'CHARLIE' out October 7! Pre-sa

In [94]:
# Contar la cantidad de colaboraciones por artista
total_colab = df_colaboracion['artist'].value_counts().reset_index()
total_colab.columns = ['artist', 'collaborations']
total_colab

,artist,collaborations
0,Juicy J,10
1,Macklemore,10
2,Bibi und Tina,10
3,DJ Khaled,9
4,Ty Dolla sign,9
...,...,...
697,Cults,1
698,MC Hariel,1
699,Prince Royce,1
700,Anitta,1


In [96]:
# Ordenar por la cantidad de colaboraciones de mayor a menor
top_10 = total_colab.head(10)
top_10

,artist,collaborations
0,Juicy J,10
1,Macklemore,10
2,Bibi und Tina,10
3,DJ Khaled,9
4,Ty Dolla sign,9
5,Wu-Tang Clan,9
6,Wale,8
7,Wizkid,8
8,Mark Ronson,8
9,B.o.B,8


In [98]:
# Obtener el nombre del artista en primer lugar y la cantidad de colaboraciones
primer_lugar = total_colab.iloc[0]
artista_primer_lugar = primer_lugar['artist']
colaboraciones_primer_lugar = primer_lugar['collaborations']

print(f"El artista que ocupa el primer lugar es {artista_primer_lugar} y ha colaborado en {colaboraciones_primer_lugar} canciones.")

El artista que ocupa el primer lugar es Juicy J y ha colaborado en 10 canciones.


In [99]:
# Obtener el artista en décimo lugar y la cantidad de colaboraciones
decimo_lugar = total_colab.iloc[9]  # Recordar que el índice es 9 para el décimo lugar (índice 0 es el primer lugar)
artista_decimo_lugar = decimo_lugar['artist']
colaboraciones_decimo_lugar = decimo_lugar['collaborations']

print(f"El artista que ocupa el décimo lugar es {artista_decimo_lugar} y ha colaborado en {colaboraciones_decimo_lugar} canciones.")

El artista que ocupa el décimo lugar es B.o.B y ha colaborado en 8 canciones.
